In [1]:
import pandas as pd
import numpy as np
import requests
from scipy.stats import norm
from scipy.optimize import bisect
from datetime import datetime, timedelta
from IPython.display import display, HTML
from scipy.stats import skew, kurtosis


In [2]:
from VolatilityModel import VolatilityModel
vm = VolatilityModel()
# from pycoingecko import CoinGeckoAPI
# cg = CoinGeckoAPI()
import cryptowatch as cw

In [3]:
import os
os.getcwd()

'/Users/virensangwan/Documents/quant/Vol Research/volresearch'

In [4]:
# os.chdir()
os.getcwd()

'/Users/virensangwan/Documents/quant/Vol Research/volresearch'

In [5]:
# import googlesheets_utility as gs
# from vol_models import get_SVI_vols, get_akima_vols

In [5]:
def Black(opt_type, F, K, r, sigma, T):
    def d1(F, K, r, sigma, T):
        return (np.log(F / K) + (sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))

    def d2(F, K, r, sigma, T):
        return d1(F, K, r, sigma, T) - (sigma * np.sqrt(T))
    
    if opt_type.lower() == "call":
        return np.exp(- r * T) * (F*norm.cdf(d1(F, K, r, sigma, T)) - K * norm.cdf(
            d2(F, K, r, sigma, T)) )
    else:
        return np.exp(- r * T) * (K*norm.cdf(-1*d2(F, K, r, sigma, T)) - F * norm.cdf(
            -1*d1(F, K, r, sigma, T)) )
    
# Helper function to solve for implied volatility
def bsaux(sigma, F, K, r, T, C, P): 
    if K < F:
        return Black('put', F, K, r, sigma, T)-P*F
    return Black('call', F, K, r, sigma, T)-C*F

In [6]:
#Deribit live data

In [13]:
products = ['BTC', 'ETH']
ivs = {'BTC':{},'ETH':{}}
for product in products:
    payload = {"currency":product,
                 "kind":"option"}

    URL = 'https://deribit.com/api/v2/public/get_book_summary_by_currency'
    response = requests.get(URL, params=payload)
    print(response)
    # print(response.text)
    print(response.url)

    data = response.json()['result']
    data[0]

    data_by_expiry = {}
    for item in data:
        name = item['instrument_name']
        curr, expiry_str, strike, opt_type = name.split('-')
        expiry = datetime.strptime(expiry_str, '%d%b%y').strftime('%Y-%m-%d')
        item.update({'expiry':expiry, 'strike':float(strike), 'opt_type':opt_type})
        data_by_expiry.setdefault(expiry, [])
        data_by_expiry[expiry].append(item)
    #%% formatting

    formatted_data = {}
    for expiry, vals in data_by_expiry.items():
        formatted_data.setdefault(expiry, {})
        # strike_data = {}
        for item in vals:
            formatted_data[expiry].setdefault(item['strike'],{})
            opt = item['opt_type']
            formatted_data[expiry][item['strike']].update({opt+'_bid':item['bid_price'], 'strike':item['strike'],
                                          opt+'_mark':item['mark_price'],
                                          opt+'_ask':item['ask_price'], opt+'_volume':item['volume'],
                                          'ul_name':item['underlying_index'],
                                          'ul':item['underlying_price']})
        formatted_data[expiry] = list(formatted_data[expiry].values())



    for expiry, chain in formatted_data.items():#if expiry=='2021-09-24':break   
        df = pd.DataFrame(chain)
        F = chain[0]['ul']
    #     df['moneyness'] = np.log(df['strike']/F)    
        T = (datetime.fromisoformat(expiry) - datetime.utcnow() + timedelta(hours=8)).total_seconds()/(365*24*60*60)
        r = 0
        strike_vols = {kdict['strike']:bisect(bsaux,-10,200,args=(F,kdict['strike'],r,T,kdict['C_mark'],kdict['P_mark']),xtol=1e-3) for kdict in chain}
    #     df['IV']=[bisect(bsaux,-10,200,args=(F,K,r,T,C,P,m),xtol=1e-3) for K,C,P,m in zip(df['strike'],df['C_mark'],df['P_mark'],df['moneyness'])]
    #     atm = estimate_atm(df, F)
    #     df['normalized_logstrike'] = df['moneyness']/(atm*np.sqrt(T))
    #     df = get_SVI_vols(df, F, atm, T, upper_lim = 1.5, lower_lim = 0.5)
        vol_params = vm.CalibrateASplineOnMoneyness(strike_vols, F, T)
        ivs[product][expiry] = vol_params

<Response [200]>
https://www.deribit.com/api/v2/public/get_book_summary_by_currency?currency=BTC&kind=option
<Response [200]>
https://www.deribit.com/api/v2/public/get_book_summary_by_currency?currency=ETH&kind=option


In [14]:
# for product in products:
btc_ivs = round(pd.DataFrame(ivs['BTC']).T.sort_index() * 100, 1)
print('\nIVS BTC')
display(btc_ivs.iloc[:,:])

eth_ivs = round(pd.DataFrame(ivs['ETH']).T.sort_index() * 100, 1)
print('\nIVS ', 'ETH')
display(eth_ivs.iloc[5:,:])


IVS BTC


,atm,-3,-2,-1.5,-1,-0.5,0.5,1,1.5,2,3
2023-07-13,31.8,167.8,141.8,130.3,119.6,111.0,99.1,108.1,118.8,130.8,151.7
2023-07-14,36.8,155.9,131.0,117.6,110.0,102.8,100.8,106.9,118.8,126.7,158.6
2023-07-15,38.3,161.5,135.9,121.5,110.7,103.5,99.7,104.3,110.9,120.9,140.5
2023-07-21,38.5,151.1,126.2,113.9,105.6,101.0,102.9,109.0,117.9,127.8,150.3
2023-07-28,39.1,155.2,129.5,116.9,107.7,101.4,103.3,109.1,116.7,127.2,146.2
2023-08-25,40.9,149.4,124.4,114.6,105.7,100.5,103.4,110.1,118.2,126.6,146.9
2023-09-29,44.2,148.5,124.6,114.4,106.5,101.2,103.7,110.1,118.3,127.0,145.4
2023-12-29,49.4,146.9,125.0,114.5,106.3,101.5,103.2,109.2,116.5,124.2,140.3
2024-03-29,51.2,NaN,126.9,115.4,106.2,100.3,103.8,110.0,117.7,125.9,139.6
2024-06-28,52.2,NaN,125.5,114.8,105.2,100.4,104.2,110.5,118.3,127.0,NaN



IVS  ETH


,atm,-3,-2,-1.5,-1,-0.5,0.5,1,1.5,2,3
2023-08-25,38.5,157.2,131.8,118.9,109.5,103.4,100.7,105.0,113.0,122.4,141.3
2023-09-29,41.5,155.0,129.6,119.5,109.5,103.1,101.2,105.4,112.8,121.5,141.7
2023-12-29,47.2,152.0,128.9,118.4,109.8,103.1,101.6,106.9,115.3,124.1,144.4
2024-03-29,49.8,151.7,131.6,119.9,110.1,103.6,102.4,107.8,115.5,124.9,142.3
2024-06-28,51.6,145.0,129.6,118.3,109.9,103.4,101.7,107.6,115.8,123.8,NaN


In [8]:
hv_days = {}
for i in btc_ivs.index:
    hv_days[i] = min(max((datetime.fromisoformat(i) - datetime.now()).days, 10), 365)

### Realized vol from Deribit API

In [229]:
payload = {"currency":'BTC',
                 }

URL = 'https://deribit.com/api/v2/public/get_historical_volatility'
response = requests.get(URL, params=payload)
print(response)
    

<Response [200]>


In [159]:
# hv = response.json()['result']

In [130]:
# len(hv)
# hv[-1]

### Realized vol from CW API

In [11]:
extreme_move_filter = 0.99 # <= 1
hv = {'BTC':{}, 'ETH':{}}
ul_moments = {'BTC':{}, 'ETH':{}}

for pdt in ['BTC', 'ETH']:
    pair = "KRAKEN:BTCUSD" if pdt == 'BTC' else "KRAKEN:ETHUSD"
    start_timestamp = int(datetime.fromisoformat('2022-06-01').timestamp())
    data = cw.markets.get(pair, ohlc=True, ohlc_after=start_timestamp, periods=["1d"])
    df_ul = pd.DataFrame(data.of_1d, columns = ['ts', 'o', 'h', 'l', 'c', 'x', 'v'])
    df_ul['cc'] = np.log(df_ul['c']/df_ul['c'].shift(1))
    extreme_moves = df_ul['cc'].abs().quantile(extreme_move_filter)
    df_ul = df_ul[df_ul['cc'] < extreme_moves]
    df_ul['gk_term1'] = 0.5 * (np.log(df_ul['h']/df_ul['l']))**2
    df_ul['gk_term2'] = (2 * np.log(2) - 1) * (np.log(df_ul['c']/df_ul['o']))**2
    
    for exp, d in hv_days.items():
        ul_moments[pdt].setdefault(exp, {})
        hv[pdt][exp] = round(100 * np.sqrt(df_ul[-d:]['gk_term1'].mean() - df_ul[-d:]['gk_term2'].mean()) * np.sqrt(365), 2)
        ul_moments[pdt][exp]['mean'] = round(np.mean(df_ul[-d:]['cc']), 4)
        ul_moments[pdt][exp]['var'] = round(np.std(df_ul[-d:]['cc'])**2, 4)
        ul_moments[pdt][exp]['skew'] = round(skew(df_ul[-d:]['cc']), 2)
        ul_moments[pdt][exp]['kurt'] = round(kurtosis(df_ul[-d:]['cc']), 2)

# print(hv, ul_moments)

In [9]:
# print(ul_moments)

In [404]:
btc_ivs['hv'] = pd.Series(hv['BTC'])
eth_ivs['hv'] = pd.Series(hv['ETH'])
btc_ivs['atm-hv'] =  btc_ivs['atm'] - btc_ivs['hv']
eth_ivs['atm-hv'] =  eth_ivs['atm'] - eth_ivs['hv']
display(btc_ivs)
display(eth_ivs)

,atm,-3,-2,-1.5,-1,-0.5,0.5,1,1.5,2,3,hv,atm-hv
2023-05-07,31.3,134.9,112.8,104.8,101.2,99.7,101.8,104.1,106.7,110.2,121.1,42.53,-11.23
2023-05-08,32.6,151.0,130.9,115.9,103.5,101.8,100.6,104.3,117.5,122.8,149.3,42.53,-9.93
2023-05-09,37.6,148.7,127.3,114.4,106.3,101.7,100.0,102.5,110.7,123.4,146.1,42.53,-4.93
2023-05-12,44.0,154.9,122.7,112.0,107.6,103.2,103.5,107.3,117.2,121.2,138.9,42.53,1.47
2023-05-19,46.1,143.7,121.1,112.7,104.3,100.5,103.8,110.3,116.1,120.9,138.9,43.71,2.39
2023-05-26,46.8,139.7,123.9,113.4,105.8,101.3,104.2,110.6,116.4,122.7,144.7,51.69,-4.89
2023-06-30,47.7,153.9,128.6,115.6,107.5,101.9,103.2,109.4,116.4,122.3,143.2,50.79,-3.09
2023-07-28,48.5,151.9,128.0,118.1,108.2,101.9,102.3,107.8,115.2,124.0,141.8,54.90,-6.40
2023-09-29,50.2,155.3,132.5,117.9,110.9,102.4,104.0,109.0,117.9,126.7,140.4,48.93,1.27
2023-12-29,52.4,162.7,130.7,118.7,107.9,101.9,103.3,109.4,116.4,122.8,NaN,51.35,1.05


,atm,-3,-2,-1.5,-1,-0.5,0.5,1,1.5,2,3,hv,atm-hv
2023-05-07,32.5,147.9,130.3,121.9,113.8,106.4,100.9,108.2,118.2,126.7,122.8,50.12,-17.62
2023-05-08,34.2,151.7,127.7,115.6,108.3,104.0,99.2,106.5,115.6,125.0,151.0,50.12,-15.92
2023-05-09,41.0,154.5,133.7,122.5,113.0,106.0,98.0,100.4,108.5,124.9,166.6,50.12,-9.12
2023-05-12,47.9,153.5,128.7,119.2,109.2,103.6,101.5,106.3,115.1,124.6,151.5,50.12,-2.22
2023-05-19,49.0,149.0,127.0,117.6,108.4,101.8,102.0,107.5,115.0,124.1,142.6,50.00,-1.00
2023-05-26,49.0,146.2,125.2,115.0,106.2,101.1,102.3,109.6,116.7,125.9,145.9,57.39,-8.39
2023-06-30,48.5,153.6,131.8,120.9,111.3,103.6,102.1,108.4,117.5,125.1,144.1,54.32,-5.82
2023-07-28,48.9,157.5,133.6,120.9,109.9,103.0,100.5,104.6,113.7,121.1,138.4,56.21,-7.31
2023-09-29,50.6,155.6,132.5,119.0,109.3,101.1,98.4,102.1,111.1,121.5,140.8,54.69,-4.09
2023-12-29,50.9,164.0,136.4,123.9,112.6,105.0,100.1,105.8,115.0,124.7,NaN,63.26,-12.36


### Forward Vol

In [61]:
btc_ivs['fv'] = 0
eth_ivs['fv'] = 0

In [170]:
expiries = btc_ivs.index

In [63]:
for i, e in enumerate(expiries):
    if i == 0: continue
    t1 = (datetime.fromisoformat(expiries[i-1]) - datetime.now()).days
    t2 = (datetime.fromisoformat(expiries[i]) - datetime.now()).days
    btc_atm1 = btc_ivs.loc[expiries[i-1],'atm']
    btc_ivs.loc[e,'fv'] = round(np.sqrt((t2*btc_ivs.loc[e,'atm']**2 - t1*(btc_atm1**2))/(t2-t1)), 1)
    eth_atm1 = eth_ivs.loc[expiries[i-1],'atm']
    eth_ivs.loc[e,'fv'] = round(np.sqrt((t2*eth_ivs.loc[e,'atm']**2 - t1*(eth_atm1**2))/(t2-t1)), 1)
btc_ivs['fv-atm'] = btc_ivs['fv'] - btc_ivs['atm']
eth_ivs['fv-atm'] = eth_ivs['fv'] - eth_ivs['atm']
btc_ivs['fv-atm'][0] = 0
eth_ivs['fv-atm'][0] = 0
# btc_ivs['fv-atm_cusum'] = btc_ivs['fv-atm'].cumsum()
# eth_ivs['fv-atm_cusum'] = eth_ivs['fv-atm'].cumsum()

In [64]:
display(btc_ivs)
display(eth_ivs)


,atm,-3,-2,-1.5,-1,-0.5,0.5,1,1.5,2,3,hv,atm-hv,fv,fv-atm
2023-04-25,45.3,140.2,125.6,117.5,109.3,102.2,98.0,104.0,114.7,121.1,121.8,38.76,6.54,0.0,0.0
2023-04-26,44.8,149.0,134.4,123.3,113.8,106.1,99.4,103.9,113.4,125.0,128.8,38.76,6.04,44.8,0.0
2023-04-28,45.8,157.4,134.6,123.2,112.0,104.1,99.9,104.2,112.0,123.3,149.1,38.76,7.04,46.3,0.5
2023-05-05,48.0,153.8,131.4,121.1,111.6,104.1,100.1,104.3,112.0,119.1,139.8,38.26,9.74,48.9,0.9
2023-05-12,49.0,156.3,132.2,121.4,111.7,104.0,99.6,103.5,109.8,117.7,133.7,37.82,11.18,50.4,1.4
2023-05-26,49.8,157.8,135.3,123.3,112.8,104.2,100.5,105.5,112.6,121.6,137.9,40.96,8.84,50.8,1.0
2023-06-30,50.9,162.2,136.3,124.0,112.5,104.0,100.6,105.2,112.6,119.9,138.1,52.99,-2.09,51.9,1.0
2023-09-29,53.3,157.8,135.2,122.6,111.4,103.4,100.8,105.2,112.0,118.7,132.6,45.10,8.20,55.0,1.7
2023-12-29,55.1,160.2,133.9,121.1,110.8,102.8,101.4,106.3,112.2,117.3,NaN,50.04,5.06,58.1,3.0
2024-03-29,55.7,NaN,NaN,118.2,110.0,103.2,102.6,107.5,113.1,119.7,NaN,57.91,-2.21,57.3,1.6


,atm,-3,-2,-1.5,-1,-0.5,0.5,1,1.5,2,3,hv,atm-hv,fv,fv-atm
2023-04-25,52.2,160.9,139.7,128.1,116.3,106.1,98.5,100.1,113.6,127.9,146.3,49.94,2.26,0.0,0.0
2023-04-26,52.1,162.5,135.5,123.9,113.7,105.1,99.9,102.5,107.1,116.0,143.3,49.94,2.16,52.1,0.0
2023-04-28,55.6,161.0,137.8,124.7,114.2,105.5,99.5,103.5,113.3,122.4,148.1,49.94,5.66,57.3,1.7
2023-05-05,56.1,157.2,134.2,122.6,112.4,104.8,99.0,101.8,108.4,116.2,138.8,47.56,8.54,56.3,0.2
2023-05-12,56.0,158.9,136.3,123.1,112.4,104.1,99.0,100.9,106.3,113.3,133.6,50.30,5.70,55.9,-0.1
2023-05-26,55.4,163.1,136.8,124.1,113.2,104.6,99.1,101.7,108.0,116.8,135.7,48.16,7.24,54.7,-0.7
2023-06-30,55.4,162.0,137.1,124.6,113.3,105.0,99.0,102.0,108.2,116.1,135.0,54.80,0.60,55.4,0.0
2023-09-29,58.1,156.9,132.1,121.0,110.9,103.8,100.3,103.8,109.6,116.7,NaN,53.73,4.37,60.0,1.9
2023-12-29,59.8,155.5,133.4,119.8,110.8,103.8,100.8,104.7,110.4,117.1,NaN,65.55,-5.75,62.6,2.8
2024-03-29,60.3,150.7,129.6,119.3,110.0,103.2,101.2,105.4,112.2,121.4,NaN,77.26,-16.96,61.6,1.3


In [65]:
btc_ivs['skew_1-1'] = btc_ivs[-1] - btc_ivs[1]
btc_ivs['skew_vol_1-1'] = btc_ivs['skew_1-1'] * btc_ivs['atm']/100
btc_ivs.round(1)

,atm,-3,-2,-1.5,-1,-0.5,0.5,1,1.5,2,3,hv,atm-hv,fv,fv-atm,skew_1-1,skew_vol_1-1
2023-04-25,45.3,140.2,125.6,117.5,109.3,102.2,98.0,104.0,114.7,121.1,121.8,38.8,6.5,0.0,0.0,5.3,2.4
2023-04-26,44.8,149.0,134.4,123.3,113.8,106.1,99.4,103.9,113.4,125.0,128.8,38.8,6.0,44.8,0.0,9.9,4.4
2023-04-28,45.8,157.4,134.6,123.2,112.0,104.1,99.9,104.2,112.0,123.3,149.1,38.8,7.0,46.3,0.5,7.8,3.6
2023-05-05,48.0,153.8,131.4,121.1,111.6,104.1,100.1,104.3,112.0,119.1,139.8,38.3,9.7,48.9,0.9,7.3,3.5
2023-05-12,49.0,156.3,132.2,121.4,111.7,104.0,99.6,103.5,109.8,117.7,133.7,37.8,11.2,50.4,1.4,8.2,4.0
2023-05-26,49.8,157.8,135.3,123.3,112.8,104.2,100.5,105.5,112.6,121.6,137.9,41.0,8.8,50.8,1.0,7.3,3.6
2023-06-30,50.9,162.2,136.3,124.0,112.5,104.0,100.6,105.2,112.6,119.9,138.1,53.0,-2.1,51.9,1.0,7.3,3.7
2023-09-29,53.3,157.8,135.2,122.6,111.4,103.4,100.8,105.2,112.0,118.7,132.6,45.1,8.2,55.0,1.7,6.2,3.3
2023-12-29,55.1,160.2,133.9,121.1,110.8,102.8,101.4,106.3,112.2,117.3,NaN,50.0,5.1,58.1,3.0,4.5,2.5
2024-03-29,55.7,NaN,NaN,118.2,110.0,103.2,102.6,107.5,113.1,119.7,NaN,57.9,-2.2,57.3,1.6,2.5,1.4


### Skewness and Kurtosis

$v_{n}(d) = σ_{n} \bigg[1 −\dfrac{γ_{1n}}{3!}d −\dfrac{γ_{2n}}{4!} (1 − d^2)\bigg]$ 

$d = \dfrac{
log(\dfrac{S_t}{K}) − (r^∗_{nt} − r_{nt})n + σ^2_n/2}{σ_n}$, let
$r = 0$, $m = \dfrac{log(\dfrac{K}{S_t})}{σ \sqrt{t}}$


$\therefore d = - m \sqrt{t} + σ_n/2$

In [28]:
def estimate_skew_iv(m, sigma, t, ul_skew, ul_kurt):
#     sigma = atm / np.sqrt(365)
    d = -m * np.sqrt(t) + sigma/2
#     d = -m + sigma * np.sqrt(t)/2 # BS d1
    iv = sigma * (1 - ul_skew * d/6 - ul_kurt * (1 - d**2)/24)
#     iv = sigma * (1 + ul_skew * (d**3 - 3*d)/6 + (ul_kurt-3) * (d**4 - 6* d**2 + 3)/24) # GC
    return iv #* np.sqrt(365)


# import numpy as np
# from scipy.stats import norm
from sklearn.neighbors import KernelDensity

# Using Kernel Density
def estimate_skew_iv2(mean, var, skewness, kurtosis, strike_price): 
    # Calculate PDF of returns using mean, variance, skewness and kurtosis
    x = np.linspace(-10, 10, 1000)
    pdf = norm.pdf(x, loc=mean, scale=np.sqrt(var))
    pdf += skewness * norm.pdf(x) * ((x - mean) / np.sqrt(var)) ** 3
    pdf += (kurtosis - 3) * norm.pdf(x) * ((x - mean) / np.sqrt(var)) ** 4

    # Estimate implied volatility using KDE
    X = x.reshape(-1, 1)
    kde = KernelDensity(kernel='gaussian', bandwidth=0.1).fit(X)
    log_dens = kde.score_samples(X)
    dens = np.exp(log_dens)
    dens /= dens.sum()
    cdf = np.cumsum(dens) * (x[1] - x[0])
    idx = np.searchsorted(cdf, strike_price)
    return x[idx]

# Using Moment Method:
from scipy.stats import norm

def estimate_skew_iv3(mean, var, skew, kurt, m, T):
    # compute standardized moments
    sigma2 = np.sqrt(var)
    sigma3 = skew * sigma2**1.5
    sigma4 = (kurt + 3) * var**2
    
    # compute option moneyness
    moneyness = m * np.sqrt(var * T) # np.log(strike / spot)
    
    # compute d1 and d2
    d1 = (moneyness + (mean + 0.5 * var) * T) / (sigma2 * np.sqrt(T))
    d2 = d1 - sigma2 * np.sqrt(T)
    
#     # compute Black-Scholes call price
#     call_price = spot * norm.cdf(d1) - strike * np.exp(-(mean - var/2) * T) * norm.cdf(d2)
    
    # compute implied volatility using equation (20)
    omega = np.sign(moneyness)
    xi = np.sqrt(np.abs(moneyness))
    numerator = np.sqrt(2 * np.pi) * omega * (xi - omega * (sigma3 / (6 * sigma2**2)) * \
                    (1 - (1 - 3/2 * omega * xi + 1/2 * omega**3 * xi**3) * norm.cdf(omega * d1) - (3/2 * \
                    omega * xi + 1/2 * omega**3 * xi**3) * norm.pdf(omega * d1) / (sigma2 * np.sqrt(T))))
    denominator = (np.exp(moneyness))**(1/2 * (1 + omega)) * (1 + (xi - omega * (sigma3 / (6 * \
                    sigma2**2))) / (1 + omega) * norm.cdf(omega * d1) + omega * (sigma3 / (\
                    6 * sigma2**2)) / (1 + omega) * norm.pdf(omega * d1) / (sigma2 * np.sqrt(T)))
    
    iv = numerator / denominator
    
    return iv



$IV ≈ σ * \sqrt(T) * \bigg[1 + (b / \sqrt(T)) * f(moneyness) + (ρ / \sqrt(T)) * skew(r)\bigg]$



In [308]:
# import numpy as np
# from scipy.stats import norm

def estimate_skew_iv6(atm, var, skewness, ul_kurtosis, moneyness, time_to_expiry):
    # Compute forward price
    BS_vol_daily = atm / np.sqrt(365) 
    SK_vol_daily = BS_vol_daily * 1 + (skewness/6)*moneyness *BS_vol_daily + ((ul_kurtosis-3)**2 /24)*(moneyness*BS_vol_daily)**2
    
    return SK_vol_daily * np.sqrt(365)


In [32]:
expiries = btc_ivs.index
skew_ivs = {'BTC':{}, 'ETH':{}}
fix_expiry = '2023-12-29'
for pdt in ['BTC', 'ETH']:
    for expiry in expiries:
        T = (datetime.fromisoformat(expiry) - datetime.utcnow() + timedelta(hours=8)).total_seconds()/(365*24*60*60)
        ul_skew = ul_moments[pdt][expiry]['skew']
        ul_kurt = ul_moments[pdt][expiry]['kurt']
        ul_mean = ul_moments[pdt][expiry]['mean']
        ul_var = ul_moments[pdt][expiry]['var']
        atm = ivs[pdt][expiry]['atm']
        m_list = list(ivs[pdt][expiry].keys())
        m_list.remove('atm')
        skew_ivs[pdt].setdefault(expiry, {'atm':atm})
        for m in m_list:
            try:
                iv_ratio = estimate_skew_iv(m, atm/20, T, ul_skew, ul_kurt-3)*20/atm
#                 iv_ratio = estimate_skew_iv6(atm, ul_var, ul_skew, ul_kurt, m, T)/atm
#                 iv_ratio = cf_iv(m, T, atm, ul_skew, ul_kurt, atm**2)/atm
                skew_ivs[pdt][expiry][m] = iv_ratio
            except:
                continue
        

/var/folders/_t/8w59m0t52b13dzxbl28j9pt80000gn/T/ipykernel_1299/511736502.py:3: RuntimeWarning: invalid value encountered in sqrt
  d = -m * np.sqrt(t) + sigma/2


In [33]:
print(ul_skew, ul_kurt)

-0.99 5.06


In [34]:
skew_ivs
btc_skew_ivs = round(pd.DataFrame(skew_ivs['BTC']).T.sort_index() * 100, 1)
print('\nIVS BTC')
display(btc_skew_ivs)

eth_skew_ivs = round(pd.DataFrame(skew_ivs['ETH']).T.sort_index() * 100, 1)
print('\nIVS ', 'ETH')
display(eth_skew_ivs)


IVS BTC


,atm,-3,-2,-1.5,-1,-0.5,0.5,1,1.5,2,3
2023-05-24,40.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-25,37.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-26,38.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-02,42.6,118.1,118.4,118.4,118.3,118.1,117.2,116.6,115.9,115.0,113.0
2023-06-09,42.8,115.6,117.0,117.3,117.3,117.0,115.7,114.6,113.3,111.7,107.6
2023-06-30,45.4,107.4,111.1,112.1,112.6,112.6,110.9,109.2,107.0,104.2,97.1
2023-07-28,45.6,94.4,104.6,108.1,110.5,112.0,111.8,110.1,107.4,103.7,93.1
2023-09-29,47.4,68.0,81.5,87.5,93.0,98.0,106.7,110.4,113.5,116.2,120.1
2023-12-29,48.5,182.7,131.2,112.4,98.3,88.9,84.1,88.8,98.1,112.1,154.0
2024-03-29,49.7,199.8,141.3,119.9,103.8,93.0,87.2,92.3,102.6,118.1,165.1



IVS  ETH


,atm,-3,-2,-1.5,-1,-0.5,0.5,1,1.5,2,3
2023-05-24,40.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-25,37.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-26,38.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-02,41.4,113.9,114.9,115.2,115.5,115.6,115.5,115.4,115.1,114.7,113.5
2023-06-09,42.7,112.7,113.5,113.6,113.5,113.1,111.9,111.0,109.8,108.5,105.1
2023-06-30,45.5,111.8,107.8,105.8,103.8,101.8,97.9,95.9,94.0,92.1,88.3
2023-07-28,45.8,106.4,107.9,108.0,107.6,106.8,103.8,101.6,99.0,95.9,88.3
2023-09-29,47.8,77.1,94.5,100.7,105.3,108.3,109.3,107.3,103.8,98.5,83.2
2023-12-29,49.8,235.8,161.5,132.8,109.9,92.6,75.1,74.8,80.3,91.4,130.7
2024-03-29,51.3,202.3,151.0,130.8,114.1,101.1,85.8,83.5,84.9,89.8,110.5


In [182]:
estimate_skew_iv3(ul_mean, ul_var, ul_skew, ul_kurt, -1, T)

/var/folders/_t/8w59m0t52b13dzxbl28j9pt80000gn/T/ipykernel_1430/1574493540.py:56: RuntimeWarning: divide by zero encountered in double_scalars
  denominator = (np.exp(moneyness))**(1/2 * (1 + omega)) * (1 + (xi - omega * (sigma3 / (6 * \
/var/folders/_t/8w59m0t52b13dzxbl28j9pt80000gn/T/ipykernel_1430/1574493540.py:57: RuntimeWarning: divide by zero encountered in double_scalars
  sigma2**2))) / (1 + omega) * norm.cdf(omega * d1) + omega * (sigma3 / (\
/var/folders/_t/8w59m0t52b13dzxbl28j9pt80000gn/T/ipykernel_1430/1574493540.py:56: RuntimeWarning: invalid value encountered in double_scalars
  denominator = (np.exp(moneyness))**(1/2 * (1 + omega)) * (1 + (xi - omega * (sigma3 / (6 * \


nan

### Vols on on Mar 25

In [225]:
display(btc_ivs)
display(eth_ivs)

,atm,-3,-2,-1.5,-1,-0.5,0.5,1,1.5,2,3,hv,atm-hv
2023-03-29,70.7,82.5,75.6,75.2,80.5,90.0,108.4,113.1,113.7,114.0,114.4,42.51,28.19
2023-03-30,59.8,127.8,119.7,113.7,105.7,100.8,105.7,114.0,123.1,NaN,NaN,42.51,17.29
2023-03-31,60.6,154.4,126.8,114.7,105.2,99.9,105.0,112.4,121.5,129.9,153.1,42.51,18.09
2023-04-07,58.0,149.1,123.8,114.3,106.1,101.0,104.1,111.1,118.9,127.1,139.1,58.51,-0.51
2023-04-14,57.5,148.5,124.5,114.3,106.2,101.3,103.2,109.0,116.1,123.6,133.2,65.96,-8.46
2023-04-28,55.8,154.9,128.6,115.3,106.2,101.1,104.5,110.9,118.5,126.9,146.2,62.30,-6.50
2023-05-26,57.4,154.2,129.0,116.5,106.8,100.5,104.7,110.8,117.9,126.6,NaN,55.10,2.30
2023-06-30,57.6,154.5,128.6,115.7,105.9,100.5,103.7,110.3,118.6,127.5,147.5,50.49,7.11
2023-09-29,58.1,155.6,130.0,117.6,107.4,100.8,103.3,110.2,119.0,128.3,NaN,49.86,8.24
2023-12-29,58.7,166.0,133.0,118.3,107.3,100.9,103.5,109.5,117.4,126.4,NaN,53.80,4.90


,atm,-3,-2,-1.5,-1,-0.5,0.5,1,1.5,2,3,hv,atm-hv
2023-03-29,58.9,79.6,94.8,91.4,89.0,92.2,110.5,122.2,133.4,142.4,146.4,47.85,11.05
2023-03-30,60.8,127.2,118.6,109.8,102.8,98.2,104.9,114.9,125.6,132.3,NaN,47.85,12.95
2023-03-31,60.8,166.7,126.8,113.8,105.5,100.5,104.8,112.7,122.2,135.0,150.8,47.85,12.95
2023-04-07,60.7,156.6,129.5,115.7,106.6,101.5,103.3,110.9,120.2,130.7,153.8,57.20,3.50
2023-04-14,62.1,153.5,128.0,115.8,106.7,101.9,103.4,109.7,118.2,127.3,145.0,60.68,1.42
2023-04-28,61.5,155.6,129.5,117.3,108.0,102.1,101.8,109.6,117.4,126.0,142.1,62.45,-0.95
2023-05-26,61.8,155.2,130.3,118.1,108.3,102.2,102.8,108.7,116.1,123.9,NaN,58.06,3.74
2023-06-30,62.2,156.5,131.4,119.6,109.6,102.7,102.2,107.9,115.4,123.3,136.8,56.88,5.32
2023-09-29,62.0,158.8,134.5,121.3,109.4,102.2,101.5,107.1,114.8,122.6,NaN,62.98,-0.98
2023-12-29,61.6,161.0,133.8,120.6,109.8,103.1,102.3,107.2,114.0,122.6,NaN,72.73,-11.13


### skew

In [186]:
df_skew = btc_ivs[[-3,-2,-1.5,-1,-0.5,0.5,1,1.5,2,3]]
benchmark = df_skew.loc['2023-04-28',:]

In [187]:
scaled_factor = {k:np.sqrt(val/45) for k,val in hv_days.items()}

In [188]:
scaled_factor

{'2023-03-14': 0.3333333333333333,
 '2023-03-15': 0.3333333333333333,
 '2023-03-17': 0.3333333333333333,
 '2023-03-24': 0.4714045207910317,
 '2023-03-31': 0.6146362971528592,
 '2023-04-28': 1.0,
 '2023-05-26': 1.2736648783028532,
 '2023-06-30': 1.5491933384829668,
 '2023-09-29': 2.1029080394116675,
 '2023-12-29': 2.5385910352879693}

In [36]:
min(False, -2)


-2